In [1]:
#Possibly have to run pip install --upgrade jupyter and pip install --upgrade ipywidgets
!pip install --upgrade jupyter
!pip install --upgrade ipywidgets
!pip install -r requirements.txt
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from sentence_transformers import SentenceTransformer, util
from PIL import Image
import torch
import pickle
import zipfile
from IPython.display import display
from IPython.display import Image as IPImage
import os
from tqdm.autonotebook import tqdm
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime
import shutil
import time
import glob
from concurrent.futures import ThreadPoolExecutor
from datetime import datetime, timedelta
import json

torch.set_num_threads(4)

model = SentenceTransformer('clip-ViT-L-14')

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.


In [2]:
# List of terms to scrape from https://relatedwords.io. 
cyber_terms = ["Cyber Security", "Cybersecurity", "Information Security"] # Limited to three for testing, but can be expanded.

# Create an empty set for cyber_terms for future querying. Set was used to prevent duplicates from being updated.
cyber_master_set = set()

# Open a request session to https://relatedwords.io. The will scrape all terms related to each intry in the cyber_terms list.
with requests.Session() as session:
    for term in cyber_terms:
        url = f"https://relatedwords.io/{term.lower()}"
        response = session.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "html.parser")
            elements_with_data_term = soup.find_all(attrs={"data-term": True}) # Identies all data-term attribute elements in the webpage HTML.
            data_term_set = {element["data-term"] for element in elements_with_data_term} # scrapes the terms inside of the data-term attribute and adds them to the data_term_set
            cyber_master_set.update(data_term_set)
        else:
            print(f"Failed to fetch the webpage for {term}")
cyber_master_list = list(cyber_master_set) #convert set to list

In [3]:
# Print the number of entries in the cyber_master_list
print(len(cyber_master_list))

# return the top 30 terms of the list
cyber_master_list = cyber_master_list[:5]

print(cyber_master_list)

974
['fencer', 'rome laboratory', 'backdoor', 'access', 'kiss and tell']


In [4]:
# def scrape_gdelt_for_date(date, hashtag):
#     url = f"https://api.gdeltproject.org/api/v2/tvv/tvv?date={date}#{hashtag}"
#     filename = f"./TXT_Files/{date}_{hashtag}.txt"
#     driver = webdriver.Edge()
#     driver.get(url)
#     wait = WebDriverWait(driver, 10)
#     container = wait.until(EC.presence_of_element_located((By.XPATH, "//div[@class='itemdetails-container']")))
#     items = container.find_elements(By.XPATH, "//div[@class='itemdetails-item']")
#     records_found = False
#     record_count = 0
    
#     # StartTimes list initailization
#     start_times = []
#     with open(filename, 'w', encoding='utf-8') as text_file:
#         for item in items:
#             link = item.find_element(By.TAG_NAME, 'a').get_attribute('href')
#             if link.startswith(f"https://api.gdeltproject.org/api/v2/tvv/tvv?id={hashtag}"):
#                 records_found = True
#                 record_count += 1
#                 image = item.find_element(By.TAG_NAME, 'img').get_attribute('src')
#                 description = item.find_element(By.TAG_NAME, 'p').text
#                 text_file.write(f"Link: {link}\nImage: {image}\nDescription: {description}\nStartTime: \n\n")
#     if records_found:
#         print(f"{record_count} records found for the requested date.")
#     else:
#         print("No records found/available for the requested date.")
#     return driver, filename

# # Extracting StartTimes from tabs
# def open_links_in_tabs(driver, filename):
#     with open(filename, "r", encoding="utf-8") as file:
#         lines = file.readlines()
#     # List for unique window handles
#     window_handles = []
#     new_start_times = []
#     for index, line in enumerate(lines):
#         if line.startswith("Link: "):
#             link = line.replace("Link: ", "").strip()
#             #This verifies if the link is already opened
#             if link not in window_handles:
#                 # Link opened in new tab
#                 driver.execute_script(f'window.open("{link}", "_blank");')
#                 window_handles.append(link)
#                 driver.switch_to.window(driver.window_handles[-1])
#                 try:
#                     wait = WebDriverWait(driver, 15)
#                     # wait for the 'imagemap' element to load
#                     thumbnails = wait.until(EC.presence_of_element_located((By.NAME, 'imagemap')))
#                     # Executing the playClip JavaScript function
#                     driver.execute_script('playClip(0);')
#                     #Timestamp extraction from the URL
#                     match = re.search(r'&play=(\d+)', driver.current_url)
#                     if match:
#                         timestamp = match.group(1)
#                         new_start_times.append(timestamp)
#                     else:
#                         timestamp = "N/A"
#                     # Find the inex of the current "StartTime: " line
#                     line_index = index
#                     while line_index >= 0 and not lines[line_index].startswith("StartTime: "):
#                         line_index -= 1
#                     # Current StartTime:" line update
#                     if line_index >= 0:
#                         lines[line_index] = f"StartTime: {timestamp}\n"
#                 except Exception as e:
#                     print(f"Error while interacting with the page: {e}")
#     for link in window_handles:
#         driver.switch_to.window(driver.window_handles[-1])
#         driver.close()
#         driver.switch_to.window(driver.window_handles[0])
#     append_start_times_to_document(new_start_times)
#     driver.quit()

# def append_start_times_to_document(start_times, document_name='./TXT_Files/start_times.txt'):
#     with open(document_name, 'a', encoding='utf-8') as file:
#         for start_time in start_times:
#             file.write(str(start_time) + '\n')
                
# def integrate_start_times_with_content(filename):
#     with open('./TXT_Files/start_times.txt', 'r') as start_time_file:
#         start_times = start_time_file.readlines()
#     with open(filename, 'r') as other_file:
#         lines = other_file.readlines()
#     final_content = []
#     start_time_index = 0
#     for line in lines:
#         if "StartTime:" in line:
#             final_content.append(line.strip() + " " + start_times[start_time_index].strip() + "\n")
#             start_time_index += 1
#         else:
#             final_content.append(line)
#     with open(filename, 'w') as other_file:
#         other_file.writelines(final_content)
#     os.remove('./TXT_Files/start_times.txt')

# user_date = input("Enter the date (in the format YYYYMMDD): ") # Restricted to one specific date for testing purposes. 
# hashtags = ["KGO", "CNN", "FOXNEWS"] # List of three US News stations available on GDELT Project. Open Source and Publicly available.
# for hashtag in hashtags:
#     driver, filename = scrape_gdelt_for_date(user_date, hashtag)
#     open_links_in_tabs(driver, filename)
#     integrate_start_times_with_content(filename)

In [5]:
def open_links_in_tabs(driver, filename):
    with open(filename, "r", encoding="utf-8") as file:
        lines = file.readlines()
    # List for unique window handles
    window_handles = []
    new_start_times = []
    for index, line in enumerate(lines):
        if line.startswith("Link: "):
            link = line.replace("Link: ", "").strip()
            #This verifies if the link is already opened
            if link not in window_handles:
                # Link opened in new tab
                driver.execute_script(f'window.open("{link}", "_blank");')
                window_handles.append(link)
                driver.switch_to.window(driver.window_handles[-1])
                try:
                    wait = WebDriverWait(driver, 15)
                    # wait for the 'imagemap' element to load
                    thumbnails = wait.until(EC.presence_of_element_located((By.NAME, 'imagemap')))
                    # Executing the playClip JavaScript function
                    driver.execute_script('playClip(0);')
                    #Timestamp extraction from the URL
                    match = re.search(r'&play=(\d+)', driver.current_url)
                    if match:
                        timestamp = match.group(1)
                        new_start_times.append(timestamp)
                    else:
                        timestamp = "N/A"
                    # Find the inex of the current "StartTime: " line
                    line_index = index
                    while line_index >= 0 and not lines[line_index].startswith("StartTime: "):
                        line_index -= 1
                    # Current StartTime:" line update
                    if line_index >= 0:
                        lines[line_index] = f"StartTime: {timestamp}\n"
                except Exception as e:
                    print(f"Error while interacting with the page: {e}")
    for link in window_handles:
        driver.switch_to.window(driver.window_handles[-1])
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
    start_times_filepath = append_start_times_to_document(new_start_times, os.path.dirname(filepath))
    driver.quit()

def append_start_times_to_document(start_times, filepath, document_name='start_times.txt'):
    start_times_filepath = os.path.join(filepath, document_name)
    with open(start_times_filepath, 'a', encoding='utf-8') as file:
        for start_time in start_times:
            file.write(str(start_time) + '\n')
    return start_times_filepath
                
def integrate_start_times_with_content(filepath, filename):
    folder_path = os.path.dirname(filepath)
    start_times_filepath = os.path.join(folder_path, 'start_times.txt')
    filename_filepath = os.path.join(folder_path, filename)
    with open(start_times_filepath, 'r', encoding='utf-8') as start_time_file:
        start_times = start_time_file.readlines()
    with open(filename_filepath, 'r', encoding='utf-8') as other_file:
        lines = other_file.readlines()
    final_content = []
    start_time_index = 0
    for line in lines:
        if "StartTime:" in line:
            final_content.append(line.strip() + " " + start_times[start_time_index].strip() + "\n")
            start_time_index += 1
        else:
            final_content.append(line)
    with open(filename_filepath, 'w', encoding='utf-8') as other_file:
        other_file.writelines(final_content)
    os.remove(start_times_filepath)
    
def download_and_extract_images(matches, hashtag, date):
    date_subfolder = os.path.join(f'{hashtag}', 'extracted_images', date)
    os.makedirs(date_subfolder, exist_ok=True)
    showid_list = []
    for match in matches:
        link, image, description, starttime_str = match
        id_match = re.search(r'id=(.*?)$', link)
        if id_match:
            showid = id_match.group(1)
            if starttime_str:
                starttime = int(starttime_str)
            else:
                starttime = None
            showid_list.append(showid)
            url = f'https://storage.googleapis.com/data.gdeltproject.org/gdeltv3/iatv/visualexplorer/{showid}.zip'
            zip_filename = os.path.join(date_subfolder, showid + '.zip')
            if not os.path.exists(zip_filename):
                response = requests.get(url, stream=True)
                if response.status_code == 200:
                    with open(zip_filename, 'wb') as f:
                        for chunk in response.iter_content(1024):
                            f.write(chunk)
            with zipfile.ZipFile(zip_filename, 'r') as zf:
                zf.extractall(date_subfolder)
            os.remove(zip_filename)
    for showid in showid_list:
        image_directory = os.path.join(date_subfolder, showid)
        img_names = list(glob.glob(os.path.join(image_directory, '*.jpg')))
        print(f"Images for showid {showid}: {len(img_names)}")
        img_emb = model.encode([Image.open(filepath) for filepath in img_names], batch_size=128, convert_to_tensor=True, show_progress_bar=True)
    return showid_list

def compute_embeddings(img_paths):
    images = [Image.open(path).convert('RGB') for path in img_paths]
    return model.encode(images, convert_to_tensor=True, show_progress_bar=True)

def search_query(query, img_paths, k=5):
    query_emb = model.encode([query], convert_to_tensor=True, show_progress_bar=True)
    img_embs = compute_embeddings(img_paths)
    hits = util.semantic_search(query_emb, img_embs, top_k=k)[0]
    results = []
    for hit in hits:
        img_path = img_paths[hit['corpus_id']]
        results.append({
            'query': query,
            'image_name': os.path.basename(img_path),
            'image_url': img_path,
            'score': hit['score'],
        })
    return results

def scrape_gdelt_for_data(date, hashtag):
    hashtag_folder = f'{hashtag}'
    os.makedirs(hashtag_folder, exist_ok=True)
    url = f"https://api.gdeltproject.org/api/v2/tvv/tvv?date={date}#{hashtag}"
    filename = f"{date}.txt"
    filepath = os.path.join(hashtag_folder, filename)
    driver = webdriver.Edge()
    driver.get(url)
    wait = WebDriverWait(driver, 10)
    container = wait.until(EC.presence_of_element_located((By.XPATH, "//div[@class='itemdetails-container']")))
    items = container.find_elements(By.XPATH, "//div[@class='itemdetails-item']")
    records_found = False
    record_count = 0
    data = []
    for item in items:
        link = item.find_element(By.TAG_NAME, 'a').get_attribute('href')
        if link.startswith(f"https://api.gdeltproject.org/api/v2/tvv/tvv?id={hashtag}"):
            records_found = True
            record_count += 1
            image = item.find_element(By.TAG_NAME, 'img').get_attribute('src')
            description = item.find_element(By.TAG_NAME, 'p').text
            data.append(f"Link: {link}\nImage: {image}\nDescription: {description}\nStartTime: \n\n")
    if records_found:
        with open(filepath, 'w', encoding='utf-8') as text_file:
            text_file.writelines(data)
        print(f"{record_count} records found for {date}.")
        with open(filepath, 'r', encoding='utf-8') as file:
            content = file.read()
        pattern = r'Link: (.*?)\nImage: (.*?)\nDescription: (.*?)\nStartTime: (.*?)\n'
        matches = re.findall(pattern, content)        
        if matches:
            download_and_extract_images(matches, hashtag, date)
    else:
        print(f"No records found/available for {date}.")
    driver.quit()
    return filepath, filename, records_found

date = input("Enter the date (in the format YYYYMMDD): ") # Restricted to one specific date for testing purposes. 
hashtags = ["CNN", "FOXNEWS"] # List of two US News stations available on GDELT Project. Open Source and Publicly available.
for hashtag in hashtags:
    filepath, filename, records_found = scrape_gdelt_for_data(date, hashtag)
    if records_found:
        driver = webdriver.Edge()
        open_links_in_tabs(driver, filepath)
        integrate_start_times_with_content(filepath, filename)
        driver.quit()
        base_folder = os.path.join(f'{hashtag}', 'extracted_images', date)
        for root, _, _ in os.walk(base_folder):
            for folder_name in os.listdir(root):
                folder_path = os.path.join(root, folder_name)
                if os.path.isdir(folder_path):
                    img_paths = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.jpg')]
                    all_results = []
                    with ThreadPoolExecutor() as executor:
                        futures = [executor.submit(search_query, query, img_paths) for query in cyber_master_list]
                        for future in futures:
                            all_results.extend(future.result())
                    pickle_folder = os.path.join(f'{hashtag}', 'pickle_folder', date)
                    os.makedirs(pickle_folder, exist_ok=True)
                    relative_folder_path = os.path.relpath(folder_path, base_folder)
                    pickle_filename = f"{hashtag}_{date}_{relative_folder_path.replace(os.sep, '_')}.pkl"
                    pickle_path = os.path.join(pickle_folder, pickle_filename)
                    with open(pickle_path, 'wb') as f:
                        pickle.dump(all_results, f)

25 records found for 20231123.
Images for showid CNNW_20231124_070000_CNN_Newsroom_Live: 915


Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Images for showid CNNW_20231124_060000_CNN_Newsroom_Live: 915


Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Images for showid CNNW_20231124_050000_CNN_Newsroom_Live: 915


...

Batches:   0%|          | 0/57 [00:00<?, ?it/s]

Batches:   0%|          | 0/57 [00:00<?, ?it/s]

Batches:   0%|          | 0/57 [00:00<?, ?it/s]

Batches:   0%|          | 0/57 [00:00<?, ?it/s]

Batches:   0%|          | 0/57 [00:00<?, ?it/s]

In [6]:
with open('./CNN/pickle_folder/20231123/CNN_20231123_CNNW_20231123_070000_CNN_NewsNight_With_Abby_Phillip.pkl', 'rb') as file:
    cnn_pkl_1 = pickle.load(file)

cnn_pkl_1

[{'query': 'fencer',
  'image_name': 'CNNW_20231123_070000_CNN_NewsNight_With_Abby_Phillip-000603.jpg',
  'image_url': 'CNN\\extracted_images\\20231123\\CNNW_20231123_070000_CNN_NewsNight_With_Abby_Phillip\\CNNW_20231123_070000_CNN_NewsNight_With_Abby_Phillip-000603.jpg',
  'score': 0.19855991005897522},
 {'query': 'fencer',
  'image_name': 'CNNW_20231123_070000_CNN_NewsNight_With_Abby_Phillip-000777.jpg',
  'image_url': 'CNN\\extracted_images\\20231123\\CNNW_20231123_070000_CNN_NewsNight_With_Abby_Phillip\\CNNW_20231123_070000_CNN_NewsNight_With_Abby_Phillip-000777.jpg',
  'score': 0.1898367702960968},
 {'query': 'fencer',
  'image_name': 'CNNW_20231123_070000_CNN_NewsNight_With_Abby_Phillip-000887.jpg',
  'image_url': 'CNN\\extracted_images\\20231123\\CNNW_20231123_070000_CNN_NewsNight_With_Abby_Phillip\\CNNW_20231123_070000_CNN_NewsNight_With_Abby_Phillip-000887.jpg',
  'score': 0.18048463761806488},
 {'query': 'fencer',
  'image_name': 'CNNW_20231123_070000_CNN_NewsNight_With_Abby_

In [7]:
with open('./FOXNEWS/pickle_folder/20231123/FOXNEWS_20231123_FOXNEWSW_20231123_070000_Hannity.pkl', 'rb') as file:
    fox_pkl_1 = pickle.load(file)

fox_pkl_1

[{'query': 'fencer',
  'image_name': 'FOXNEWSW_20231123_070000_Hannity-000907.jpg',
  'image_url': 'FOXNEWS\\extracted_images\\20231123\\FOXNEWSW_20231123_070000_Hannity\\FOXNEWSW_20231123_070000_Hannity-000907.jpg',
  'score': 0.18771319091320038},
 {'query': 'fencer',
  'image_name': 'FOXNEWSW_20231123_070000_Hannity-000347.jpg',
  'image_url': 'FOXNEWS\\extracted_images\\20231123\\FOXNEWSW_20231123_070000_Hannity\\FOXNEWSW_20231123_070000_Hannity-000347.jpg',
  'score': 0.16577032208442688},
 {'query': 'fencer',
  'image_name': 'FOXNEWSW_20231123_070000_Hannity-000332.jpg',
  'image_url': 'FOXNEWS\\extracted_images\\20231123\\FOXNEWSW_20231123_070000_Hannity\\FOXNEWSW_20231123_070000_Hannity-000332.jpg',
  'score': 0.16093285381793976},
 {'query': 'fencer',
  'image_name': 'FOXNEWSW_20231123_070000_Hannity-000335.jpg',
  'image_url': 'FOXNEWS\\extracted_images\\20231123\\FOXNEWSW_20231123_070000_Hannity\\FOXNEWSW_20231123_070000_Hannity-000335.jpg',
  'score': 0.1477425992488861},
 